# 概要
* 複数インデックスのマージにいよいよトライする
* treeは自動で要約つけてくれるの？？めっちゃ便利やん
* 訳としては完璧やん...
* num_outputは2048も対応している→ダイでエラー発生
* 親の要約が変なのが惜しい
* b_indexがおかしい
* #prompt_helperで重複を許すようにすれば自然な日本語になるはず#！！
* そもそもこういう問題があるからチュートリアルはkeyword_tableを使っていたのか
* indexを作っただけではembeddingしていないことに注意！！

# 参考
* そもそもtree構造が前提なのでtreeのindexを確認する
* https://gpt-index.readthedocs.io/en/latest/how_to/composability.html
* https://note.com/npaka/n/n5d4ff4380055
* https://note.com/oshizo/n/n137aaa2c29d4
* 日本語としてparseされない...

# 結局
* 一応動いてる、細かいところまでとれているかはわからないが。

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [5]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

#documents = SimpleDirectoryReader('story/dai').load_data()
documents = SimpleDirectoryReader('story/dragonball').load_data()

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024+512
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
#d_index = GPTSimpleVectorIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)
#b_index = GPTTreeIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1


# keyword_table index

In [7]:
kt_index = GPTKeywordTableIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> Adding chunk: 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た...
DEBUG:root:> Adding chunk: 

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8...
DEBUG:root:> Adding chunk: 悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き...
DEBUG:root:> Adding chunk: 

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過...
DEBUG:root:> Adding chunk: 仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と...
DEBUG:root:> Adding chunk: フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落と...
DEBUG:root:> Adding chunk: トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空に...
DEBUG:root:> Adding chunk: 悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強...
DEBUG:root:> Adding chunk: 天下一武道会の最中、悟空たちは界王よりもさらに高位の存在である界王神から、恐ろしい力を持つ魔人...
DEBUG:root:> Adding chunk: ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords 

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2729 request_id=46dca308111e0004097da3bcdbdaae34 response_code=200
DEBUG:root:
KEYWORDS: 悟空, ヤジロベー, 仙猫カリン, ピッコロ大魔王, 神龍, 仲間, 天界, 修行, 青年, 牛魔王, チチ, マジュニア, 決勝戦, 優勝
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\n\\n\\u30b5\\u30a4\\u30e4\\u4eba\\u7de8\\n\\u30d4\\u30c3\\u30b3\\u30ed\\uff08\\u30de\\u30b8\\u30e5\\u30cb\\u30a2\\uff09\\u3068\\u306e\\u95d8\\u3044\\u304b\\u3089\\u7d045\\u5e74\\u5f8c\\u3001\\u606f\\u5b50\\u306e\\u5b6b\\u609f\\u98ef\\u3092\\u5132\\u3051\\u3066\\u5e73\\u548c\\u306a\\u65e5\\u3005\\u3092\\u904e\\u3054\\u3057\\u3066\\u3044\\u305f\\u609f\\u7a7a\\u306e\

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2286 request_id=12a081e8eedf71d9bc66880d7d0d3c82 response_code=200
DEBUG:root:
KEYWORDS: フリーザ, ベジータ, ピッコロ, 悟空, 伝説, 超サイヤ人, ポルンガ, 地球, ドラゴンボール, 蘇生
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\u30c8\\u30e9\\u30f3\\u30af\\u30b9\\u3068\\u540d\\u4e57\\u308b\\u305d\\u306e\\u9752\\u5e74\\u306f\\u3001\\u81ea\\u5206\\u306f\\u672a\\u6765\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u30d6\\u30eb\\u30de\\u3068\\u30d9\\u30b8\\u30fc\\u30bf\\u306e\\u606f\\u5b50\\u3067\\u3042\\u308b\\u3053\\u3068\\u3092\\u609f\\u7a7a\\u306b\\u3060\\u3051\\u660e\\u304b\\u3059\\u3068\\u540c\\u664

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2374 request_id=b7213df95c99ae0f847d79ab7767611c response_code=200
DEBUG:root:
KEYWORDS: 悟空, 悟飯, ベジータ, 界王神, 魔人ブウ, 戦士, フュージョン, 融合, 潜在能力, 吸収
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u5354\\u529b\\u3082\\u3042\\u308a\\u3001\\u5730\\u7403\\u30fb\\u30ca\\u30e1\\u30c3\\u30af\\u661f\\u30fb\\u3042\\u306e\\u4e16\\u306e\\u4eba\\u3005\\u306e\\u30a8\\u30cd\\u30eb\\u30ae\\u30fc\\u306b\\u3088\\u3063\\u3066\\u4f5c\\u308a\\u4e0a\\u3052\\u3089\\u308c\\u305f\\u8d85\\u7279\\u5927\\u306e\\u5143\\u6c17\\u7389\\u306b\\u3088\\u3

In [8]:
dir(kt_index)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_document_to_index',
 '_async_add_document_to_index',
 '_async_extract_keywords',
 '_build_fallback_text_splitter',
 '_build_index_from_documents',
 '_delete',
 '_docstore',
 '_embed_model',
 '_extract_keywords',
 '_get_nodes_from_document',
 '_include_extra_info',
 '_index_registry',
 '_index_struct',
 '_insert',
 '_is_protocol',
 '_llm_predictor',
 '_preprocess_query',
 '_process_documents',
 '_prompt_helper',
 '_text_splitter',
 '_update_index_registry_and_docstore',
 '_use_async',
 '_validate_documents',

In [15]:
kt_index.docstore.docs.keys()

dict_keys(['20504282-176d-4932-b1e7-73e420235223'])

In [20]:
kt_index.docstore.docs['20504282-176d-4932-b1e7-73e420235223']

KeywordTable(text=None, doc_id='20504282-176d-4932-b1e7-73e420235223', embedding=None, extra_info=None, table={'ヤムチャ': {6201837512700076671}, '盗賊': {6201837512700076671}, 'ウーロン': {6201837512700076671}, '孫悟空': {6201837512700076671}, 'ドラゴンボール': {6201837512700076671, 7510331091196385733, 8192120892737752775, 7239158803188884780, 4880187643955340463, 970764164167309241}, 'ブルマ': {8817195983466389445, 6201837512700076671, 7239158803188884780}, '少年': {6201837512700076671}, 'ピラフ': {6201837512700076671}, '少女': {6201837512700076671}, '四星球': {6201837512700076671}, '悟空': {7510331091196385733, 8817195983466389445, 8192120892737752775, 7239158803188884780, 4880187643955340463, 6158634650112249680, 8614812770996273364, 970764164167309241, 9022682885623469335}, '天津飯': {6158634650112249680}, '孫悟飯': {8192120892737752775, 6158634650112249680}, '修行': {4880187643955340463, 6158634650112249680, 970764164167309241, 8614812770996273364}, 'クリリン': {6158634650112249680, 7239158803188884780}, 'レッドリボン軍': {61586346

In [21]:
dir(kt_index.docstore.docs['20504282-176d-4932-b1e7-73e420235223'])

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_get_index',
 'add_node',
 'dataclass_json_config',
 'doc_id',
 'embedding',
 'extra_info',
 'extra_info_str',
 'from_dict',
 'from_json',
 'get_doc_id',
 'get_embedding',
 'get_text',
 'get_texts',
 'get_type',
 'get_types',
 'is_doc_id_none',
 'keywords',
 'schema',
 'size',
 'table',
 'text',
 'text_chunks',
 'to_dict',
 'to_json']

In [23]:
kt_index.docstore.docs['20504282-176d-4932-b1e7-73e420235223'].table

{'ヤムチャ': {6201837512700076671},
 '盗賊': {6201837512700076671},
 'ウーロン': {6201837512700076671},
 '孫悟空': {6201837512700076671},
 'ドラゴンボール': {970764164167309241,
  4880187643955340463,
  6201837512700076671,
  7239158803188884780,
  7510331091196385733,
  8192120892737752775},
 'ブルマ': {6201837512700076671, 7239158803188884780, 8817195983466389445},
 '少年': {6201837512700076671},
 'ピラフ': {6201837512700076671},
 '少女': {6201837512700076671},
 '四星球': {6201837512700076671},
 '悟空': {970764164167309241,
  4880187643955340463,
  6158634650112249680,
  7239158803188884780,
  7510331091196385733,
  8192120892737752775,
  8614812770996273364,
  8817195983466389445,
  9022682885623469335},
 '天津飯': {6158634650112249680},
 '孫悟飯': {6158634650112249680, 8192120892737752775},
 '修行': {970764164167309241,
  4880187643955340463,
  6158634650112249680,
  8614812770996273364},
 'クリリン': {6158634650112249680, 7239158803188884780},
 'レッドリボン軍': {6158634650112249680},
 '天下一武道会': {970764164167309241, 48801876439553404

In [51]:
kt_index.query("セルは最後どうなった？")

INFO:root:> Starting query: セルは最後どうなった？
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30bb\\u30eb\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\uff1f\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API

Response(response='\nセルは、悟空が最終的に自分を犠牲にしてセルを消滅させることで最後に消滅した。', source_nodes=[SourceNode(source_text='トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空にだけ明かすと同時に、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる。その後本当に心臓病によって危篤状態になるも、トランクスから渡された特効薬によって生還、来るべき日に備えて3年間各々に修行してその日を迎える悟空たちであったが、事態はトランクスが知っている歴史とは大きく違うものとなり、彼さえ知らなかった人造人間たちまで現れ、さらに究極の人造人間・セルが未来から出現。悟空らの想定を遥かに超えた戦士が続々と現れた。\n\n人造人間17号と人造人間18号を吸収することで完全体となったセルは地球の命運を賭けた武道会「セルゲーム」の開催を全世界に宣言する', doc_id='780cc593-3d8b-4baf-83e1-0fe8b9ae90de', extra_info=None, node_info={'start': 1915, 'end': 2260}, similarity=None)], extra_info=None)

In [24]:
kt_index.docstore.docs['20504282-176d-4932-b1e7-73e420235223'].text_chunks

{6201837512700076671: Node(text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する', doc_id='a75ce452-c022-4ab5-9344-e9759062d9d2', embedding=None, extra_info=None, index=0, child_indices=set(), ref_doc_id='780cc593-3d8b-4baf-83e1-0fe8b9ae90de', node_info={'start': 0, 'end': 296}),
 6158634650112249680: Node(text='\n\nその後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。\n\nピッコロ大魔王編\n天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される', doc_id='5382905f-d399-4743-9de4-687bb8881953', embedding=None, extra_info=None, index=1, ch

In [26]:
kt_index.docstore.docs['20504282-176d-4932-b1e7-73e420235223'].embedding

In [27]:
d_index = GPTTreeIndex.load_from_disk('dai_tree.json', prompt_helper=prompt_helper, llm_predictor=llm_predictor)
b_index = GPTTreeIndex.load_from_disk('dragon_tree.json', prompt_helper=prompt_helper, llm_predictor=llm_predictor)

In [28]:
d_index.docstore.docs.keys()

dict_keys(['38f1d4b1-3314-4635-a146-82951b12f846'])

# all_indexの検証

In [40]:
all_index = GPTKeywordTableIndex([b_index, d_index], max_keywords_per_chunk=15)

DEBUG:root:> Adding chunk: 
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 15 keywords from the text. Avoid stopwords.---------------------\\n\\n\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u304c\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3044\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u30927\\u3064\\u96c6\\u3081\\u308b\\u3053\\u3068\\u3067\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u308b\\u3053\\u3068\\u3092\\u77e5\\u308a\\u3001\\u6b8b\\u308a\\u306e\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u3092\\u63a2\\u3059\\u65c5\\u306

DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=7766 request_id=99475ac4fc6c8fac3e3a7b5fc261b219 response_code=200
DEBUG:root:
KEYWORDS: 孫悟空, 少年, ブルマ, ドラゴンボール, 神龍, ウーロン, ピラフ, 亀仙人, 修行, 天下一武道会, ピッコロ大魔王, クリリン, サイヤ人, ナッパ, ベジータ, フリーザ, トランクス, セルゲーム, マジュニア, ポルンガ, 悟飯, 超サイヤ人, 占いババ, 界王神, 魔人ブウ, 元気玉, ウーブ
DEBUG:root:> Adding chunk: 
十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 15 keywords from the text. Avoid stopwords.---------------------\\n\\n\\u5341\\u6570\\u5e74\\u524d\\u3001\\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u7387\\u3044\\u308b\\u9b54\\u738b\\u8ecd\\u3068\

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3030 request_id=1666f7253ca27560f160d28cb660c750 response_code=200
DEBUG:root:
KEYWORDS: ハドラー, モンスター, ダイ, アバン, ポップ, マホカトール, バラン, メガンテ, ロモス, パプニカ, 鬼岩城, バーン, ミナカトール, カール, 鬼眼
INFO:root:> [build_index_from_documents] Total LLM token usage: 4771 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 0 tokens


In [47]:
all_index.docstore.docs['57959422-1c27-44a0-b8d8-0c803977ba44'].table

{'孫悟空': {4347119370851057834},
 'クリリン': {4347119370851057834},
 '界王神': {4347119370851057834},
 'ウーブ': {4347119370851057834},
 'トランクス': {4347119370851057834},
 'フリーザ': {4347119370851057834},
 '悟飯': {4347119370851057834},
 'ウーロン': {4347119370851057834},
 '神龍': {4347119370851057834},
 'セルゲーム': {4347119370851057834},
 '修行': {4347119370851057834},
 'マジュニア': {4347119370851057834},
 '魔人ブウ': {4347119370851057834},
 'ピラフ': {4347119370851057834},
 'ブルマ': {4347119370851057834},
 '亀仙人': {4347119370851057834},
 'サイヤ人': {4347119370851057834},
 'ベジータ': {4347119370851057834},
 'ナッパ': {4347119370851057834},
 '占いババ': {4347119370851057834},
 '元気玉': {4347119370851057834},
 'ポルンガ': {4347119370851057834},
 '超サイヤ人': {4347119370851057834},
 'ドラゴンボール': {4347119370851057834},
 '天下一武道会': {4347119370851057834},
 'ピッコロ大魔王': {4347119370851057834},
 '少年': {4347119370851057834},
 '鬼眼': {4545755552743438823},
 'アバン': {4545755552743438823},
 'ハドラー': {4545755552743438823},
 'ポップ': {4545755552743438823},
 'バーン': {4545755

In [45]:
all_index.docstore.docs.keys()

dict_keys(['8618eeae-fc69-4287-b646-83b56c5ac565', '38f1d4b1-3314-4635-a146-82951b12f846', '57959422-1c27-44a0-b8d8-0c803977ba44'])

##  重要
all_indexはListIndexが各ノードのreferenceを保持している

In [48]:
all_index.query("ダイは最後どうなりましたか？日本語で答えてください。")

INFO:root:> Starting query: ダイは最後どうなりましたか？日本語で答えてください。
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30c0\\u30a4\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u308a\\u307e\\u3057\\u305f\\u304b\\uff1f\\u65e5\\u672c\\u8a9e\\u3067\\u7b54\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 256, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urll

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3241 request_id=01ab8f7ce81d0a2e395c436273214288 response_code=200
DEBUG:root:
ダイは最後、黒の核晶を空高く運ぶために爆発の閃光の中に消えてしまいました。
DEBUG:root:> Initial response: 
ダイは最後、黒の核晶を空高く運ぶために爆発の閃光の中に消えてしまいました。
INFO:root:> [query] Total LLM token usage: 2217 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


Response(response='\nダイは最後、黒の核晶を空高く運ぶために爆発の閃光の中に消えてしまいました。', source_nodes=[SourceNode(source_text='\n十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。それから十数年後、デルムリン島に住む少年ダイは、勇者になることを夢見ていた。ある日、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受けて、短期間のうちに秘められた力を開花させていく。15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲するが、ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。ダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立ち、ロモス王国とパプニカ王国を占拠した魔王軍六大軍団と戦い、勝利を収める。\nダイは人間からの視線に苦悩していたが、その前に竜騎将バランが現れ、ダイが神々が作った伝説の存在「竜の騎士」であることを告げた。バランはダイに魔王軍に加わるよう促したが、ダイは拒否した。その後、ポップがメガンテを発動して犠牲になり、ダイの記憶が蘇った。レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も新たな力を蓄え、ハドラーは妖魔司教ザボエラの手で自らを超魔生物にさせる。ダイ一行は魔王軍の本拠地である鬼岩城を破壊したが、バーンパレスには魔宮の門によって閉ざされていた。バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラ

In [61]:
all_index.query("魔人ブウはどのように倒されましたか？、日本語で答えてください。")

INFO:root:> Starting query: 魔人ブウはどのように倒されましたか？、日本語で答えてください。
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u9b54\\u4eba\\u30d6\\u30a6\\u306f\\u3069\\u306e\\u3088\\u3046\\u306b\\u5012\\u3055\\u308c\\u307e\\u3057\\u305f\\u304b\\uff1f\\u3001\\u65e5\\u672c\\u8a9e\\u3067\\u7b54\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 256, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropp

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3392 request_id=be743e870c67747ff30e440f99eba0e9 response_code=200
DEBUG:root:
ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅した。
DEBUG:root:> Initial response: 
ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅した。
INFO:root:> [query] Total LLM token usage: 2734 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


Response(response='\nドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅した。', source_nodes=[SourceNode(source_text="\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。サイヤ人編では、ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。\n悟空は孫悟飯を助けるため、宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒す。その後、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。フリーザを倒し、地球の神と殺された仲

### 階層構造の場合は、modeは再帰にし、詳細な条件はquery_configsで指定する

In [147]:
# set query config
query_configs = [
    {
        "index_struct_type": "keyword_table",
        "query_mode": "default",
        "query_kwargs": {
            "child_branch_factor": 2
        }        
    }
]

In [94]:
from llama_index.prompts.prompts import QuestionAnswerPrompt
default_prompt="""
文脈情報は以下です。
---
{context_str}
---
事前知識ではなく、文脈情報を参考に日本語で質問に答えてください。：{query_str}
"""

custom_prompt = QuestionAnswerPrompt(template=default_prompt)#), "constext_str", "query_str")

In [98]:
#  recursiveではquery_configs 必須

default_prompt="""
文脈情報は以下です。
---
{context_str}
---
事前知識ではなく、文脈情報を参考に質問に答えてください。：{query_str}
"""

all_index.query("魔人ブウ", mode="recursive", query_configs=query_configs)#, text_qa_template=custom_prompt)

INFO:root:> Starting query: 魔人ブウ
INFO:root:query keywords: ['魔人ブウ']
INFO:root:> Extracted keywords: ['魔人ブウ']
INFO:root:> [query] Total LLM token usage: 0 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


Response(response=None, source_nodes=[], extra_info=None)

In [99]:
from llama_index.composability import ComposableGraph

In [100]:
graph = ComposableGraph.build_from_index(all_index)

In [112]:
response = graph.query("魔人ブウはどのように倒されたのか日本語で教えて")

INFO:root:> Starting query: 魔人ブウはどのように倒されたのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u9b54\\u4eba\\u30d6\\u30a6\\u306f\\u3069\\u306e\\u3088\\u3046\\u306b\\u5012\\u3055\\u308c\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 256, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 548
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2768 request_id=a754b95686d25928b198573853b2eba5 response_code=200
DEBUG:root:
ANSWER: 2 
This summary was selected because it provides the most detailed information about how Majin Buu was defeated. It explains that Goku and Vegeta faced Majin Buu in a final battle in the Kaioshin Realm, and with the help of the Dragon Balls, they were able to defeat him.
DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 2),where each item in the list corresponds to a summary.
---------------------
(1)  孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めると神龍（シェンロン）が現れるということを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、天下一武道会に出場し準優勝を果たす。3年後の天下一武道会では、鶴仙流の天津飯（てんしんはん）と

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 686
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3356 request_id=c00dbe7d1c9fed1c6a292728e88b07be response_code=200
DEBUG:root:
ANSWER: 8 
This summary was selected because it provides the most detailed description of how Majin Buu was defeated. It explains that he was defeated by the combined efforts of the Dragon Balls, the energy of the people of Earth, Namakk and the Other World, and the Super Genki Dama created by Goku and Vegeta. It also mentions that Goku and Vegeta returned to the present world to save their friends from Majin Buu.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 9),where each item in the list corresponds to a summary.
---------------------
(1) 怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザに勝利する。ポルンガによって地球に帰還した悟飯たちは復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇

In [142]:
response = graph.query("ハドラーを倒したのは誰か日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: ハドラーを倒したのは誰か日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30cf\\u30c9\\u30e9\\u30fc\\u3092\\u5012\\u3057\\u305f\\u306e\\u306f\\u8ab0\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 256, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 720
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3464 request_id=3b4f2ad3acfd6bc4fa63278f03f2ec10 response_code=200
DEBUG:root:
ANSWER: 3 
This summary was selected because it provides the most detailed information about the character who defeated the villain, Harudora. It states that Dai, with the help of his companions and the power of the mysterious crest on his forehead, was able to overpower Harudora and restore peace to the world. It also mentions the sacrifice of Baran, who used all of his power to protect Dai and his companions from the explosion of the black core.
DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1739 request_id=1f5b22495cd8ffee68af0dd3149ee58a response_code=200
DEBUG:root:
ANSWER: 3
This summary was selected because it mentions that "ダイはハドラーを倒す" (Daichi defeats Haldor).
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 9),where each item in the list corresponds to a summary.
---------------------
(1) その後に現れたバーンとの直接対決において、ダイ一行は圧倒的な力の差の前に敗北するが、捨て駒にされたハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。  最終決戦 邪魔者を退けたバーンはバーンパレスを飛行させ、世界各地への爆撃を実行する

(2) 一方、カール王国女王のフローラの協力も得て辛うじて生き延びたダイ一行は、5人目のアバンの使徒になったレオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にアバンの使徒との決着を願うハドラーと親衛騎団も乱入し、激闘を繰り広げる

(3) ダイはハドラーを倒すが、その直後にキルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった

(4)   父・バランから受け継いだ竜の紋章の力でダイはバーンを追い詰めるが、バーンはミストバーンに預けていた肉体を使って真の姿

In [143]:
response = graph.query("ハドラーは最後どうなったのか日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: ハドラーは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30cf\\u30c9\\u30e9\\u30fc\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 256, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenA

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 671
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3665 request_id=d8dfe644cc64172c88870997aba366bc response_code=200
DEBUG:root:
ANSWER: 3 
This summary was selected because it provides the most detailed information about the fate of the character, Lord Hades. It explains how he was defeated by the protagonist, Dai, and how he attempted to escape to the Demon Realm. It also explains how Dai sacrificed himself to save his friends, and how his sword, with its magical gem, continued to shine, indicating that Dai was still alive.
DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。それから十数年後、デルムリン島に住む少年ダイは、勇者になることを夢見て

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 587
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2906 request_id=f8d5a1f1befccd3b968e3024cb0b6fa0 response_code=200
DEBUG:root:
ANSWER: 2
This summary was selected because it mentions that Handlar joined the fight against Burn and attempted to settle the conflict with the Apostles of Avan. It also mentions that Handlar was part of the group that infiltrated Burn Palace, which is relevant to the question of what happened to Handlar in the end.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 9),where each item in the list corresponds to a summary.
---------------------
(1) その後に現れたバーンとの直接対決において、ダイ一行は圧倒的な力の差の前に敗北するが、捨て駒にされたハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。  最終決戦 邪魔者を退けたバーンはバーンパレスを飛行させ、世界各地への爆撃を実行する

(2) 一方、カール王国女王のフローラの協力も得て辛うじて生き延びたダイ一行は、5人目のアバンの使徒になったレオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの

In [135]:
response.get_formatted_sources()

'> Source (Doc id: 38f1d4b1-3314-4635-a146-82951b12f846): \n十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。それから十数年後、デ...\n\n> Source (Doc id: 4c0af764-8c18-4260-9cf1-6ec986556b97): 一方、カール王国女王のフローラの協力も得て辛うじて生き延びたダイ一行は、5人目のアバンの使徒になったレオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にアバンの...'

In [148]:
response = graph.query("ハドラーは最後どうなったのか日本語で教えて",query_configs=query_configs)

TypeError: BaseGPTIndexQuery.__init__() got an unexpected keyword argument 'child_branch_factor'

In [131]:
response = graph.query("ハドラーは最後どうなったのか日本語で教えて")

INFO:root:> Starting query: ハドラーは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30cf\\u30c9\\u30e9\\u30fc\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 256, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenA

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 697
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3772 request_id=46031991b1862e0f7f9ede26b5fe6ca8 response_code=200
DEBUG:root:
ANSWER: 3 
This summary was selected because it provides the most detailed information about the fate of the character, Lord Hades. It explains how he was defeated by the protagonist, Dai, and how he attempted to escape to the Demon Realm. It also explains how Dai and Pop carried the black core to the sky, and how Dai disappeared in a flash of light. Finally, it explains how Pop vowed to protect the world until Dai's return.
DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。それから十数年後、

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 587
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2787 request_id=8b551cfcf7c49725d90f9e6c96f34710 response_code=200
DEBUG:root:
ANSWER: 2
This summary was selected because it mentions that Handlar joined the fight against Burn and attempted to settle the conflict with the Apostles of Avan. It also mentions that Handlar was part of the group that infiltrated Burn Palace, which is relevant to the question of what happened to Handlar in the end.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 9),where each item in the list corresponds to a summary.
---------------------
(1) その後に現れたバーンとの直接対決において、ダイ一行は圧倒的な力の差の前に敗北するが、捨て駒にされたハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。  最終決戦 邪魔者を退けたバーンはバーンパレスを飛行させ、世界各地への爆撃を実行する

(2) 一方、カール王国女王のフローラの協力も得て辛うじて生き延びたダイ一行は、5人目のアバンの使徒になったレオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの

In [132]:
response.get_formatted_sources()

'> Source (Doc id: 38f1d4b1-3314-4635-a146-82951b12f846): \n十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。それから十数年後、デ...\n\n> Source (Doc id: 4c0af764-8c18-4260-9cf1-6ec986556b97): 一方、カール王国女王のフローラの協力も得て辛うじて生き延びたダイ一行は、5人目のアバンの使徒になったレオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にアバンの...'